# Image classification example: cats vs dogs

Learn how to use Keras and TensorFlow

In [1]:
import tensorflow as tf
from tensorflow import keras

Learn about:

- Tensors, variables, and gradients in TensorFlow
- Creating layers by subclassing the Layer class
- Writing low-level training loops
- Tracking losses created by layers via the add_loss() method
- Tracking metrics in a low-level training loop
- Speeding up execution with a compiled tf.function
- Executing layers in training or inference mode
- The Keras Functional API

## Tensors

TensorFlow is an infrastructure layer for differentiable programming. At its heart, it's a framework for manipulating N-dimensional arrays (tensors), much like NumPy.

However, there are three key differences between NumPy and TensorFlow:

1. TensorFlow can leverage hardware accelerators such as GPUs and TPUs.
2. TensorFlow can automatically compute the gradient of arbitrary differentiable tensor expressions.
3. TensorFlow computation can be distributed to large numbers of devices on a single machine, and large number of machines (potentially with multiple devices each).

In [2]:
constant_tensor = tf.constant([[1, 0], [0, 1]])
constant_tensor

2022-03-27 17:16:16.917244: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1, 0],
       [0, 1]], dtype=int32)>

You can get the tensor's values as a Numpy nd-array.

In [3]:
constant_tensor.numpy()

array([[1, 0],
       [0, 1]], dtype=int32)

Find its type and shape the same way of Numpy arrays.

In [4]:
print(f'DType: {constant_tensor.dtype}')
print(f'Shape: {constant_tensor.shape}')

DType: <dtype: 'int32'>
Shape: (2, 2)


Constant tensors of 1s and 0s.

In [5]:
tf.ones(shape=(3, 2))


<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32)>

In [6]:
tf.zeros(shape=(2, 3))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0., 0., 0.],
       [0., 0., 0.]], dtype=float32)>

Random uniform tensor.

In [7]:
tf.random.uniform(shape=(3, 1), minval=0., maxval=1.)

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[0.22078836],
       [0.7939428 ],
       [0.27340877]], dtype=float32)>

Random from normal distribution.

In [8]:
tf.random.normal(shape=(1, 3), mean=0., stddev=1.)

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-0.7298222 ,  0.7450754 ,  0.06464498]], dtype=float32)>

## Variables

Variables are special tensors used to store mutable state (such as the weights of a neural network). You create a variable using some initial value.

In [9]:
gaussian_tensor = tf.random.normal(shape=(1, 10))
variable = tf.Variable(gaussian_tensor)
print(variable)

<tf.Variable 'Variable:0' shape=(1, 10) dtype=float32, numpy=
array([[-0.6311023 ,  1.3034041 ,  0.40409243, -0.6539666 ,  2.9951575 ,
        -0.33104387, -1.1689478 ,  1.346186  ,  0.17106554, -1.6727511 ]],
      dtype=float32)>


To update a variable just use the <code>sub/add/assign</code> methods. Keep in mind that like arrays and matrixes the shape must be coherent to allow operations.

In [10]:
# assign new tensor to variable
new_gaussian_tensor = tf.random.normal(shape=(1, 10))
variable.assign(new_gaussian_tensor)
# verify the assign operation is true
for i in range(new_gaussian_tensor.shape[0]):
    for j in range(new_gaussian_tensor.shape[1]):
        assert new_gaussian_tensor[i, j] == new_gaussian_tensor[i, j]

# add tensor to variable
added_uniform_tensor = tf.random.uniform(shape=(1, 10))
variable.assign_add(added_uniform_tensor)
# verify the assign + add operation is true
for i in range(new_gaussian_tensor.shape[0]):
    for j in range(new_gaussian_tensor.shape[1]):
        assert variable[i, j] == new_gaussian_tensor[i, j] + added_uniform_tensor[i, j]

# subtract tensor to variable
subbed_uniform_tensor = tf.random.uniform(shape=(1, 10))
variable.assign_sub(subbed_uniform_tensor)
# verify the assign + add + sub operation is true
for i in range(new_gaussian_tensor.shape[0]):
    for j in range(new_gaussian_tensor.shape[1]):
        assert variable[i, j] == new_gaussian_tensor[i, j] + added_uniform_tensor[i, j] - subbed_uniform_tensor[i, j]

## Doing math in TensorFlow

If you've used NumPy, doing math in TensorFlow will look very familiar. The main difference is that your TensorFlow code can run on GPU and TPU.

In [14]:
a = tf.random.normal(shape=(2, 2))
b = tf.random.normal(shape=(2, 2))

# sum
c = a + b
# subtraction
d = a - b
# moltiplication
e = a * b
# division
f = a / b
# square
g = tf.square(c)
# exponential
h = tf.exp(c)

c, d, e, f, g, h

(<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[-0.02320025, -2.023105  ],
        [-0.6110183 , -0.7449284 ]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[ 0.24314761, -0.24281383],
        [-0.07221237, -0.2045129 ]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[-0.01464563,  1.0084987 ],
        [ 0.09203219,  0.1282732 ]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[-0.82578987,  1.27278   ],
        [ 1.2680459 ,  1.7568728 ]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[5.3825165e-04, 4.0929537e+00],
        [3.7334335e-01, 5.5491835e-01]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[0.97706676, 0.13224421],
        [0.54279786, 0.47476828]], dtype=float32)>)